# Using hcubature on uniform mesh of rectangular elements

## Import Packages

In [50]:
using ForwardDiff
using QuadGK
using HCubature 
using StaticArrays 
using LinearAlgebra
using BenchmarkTools
using Plots

In [4]:
# a point in 3D is a tuple of 3 coordinates 
# we here introduce static vectors that appear to be vital to reduce the number of allocations 
const Point3D = SVector{3,Float64};

In [88]:
function hcubature_count(f, a, b; kws...)
    count = 0
    i = hcubature(a, b; kws...) do x
        count += 1
        # display(count)
        f(x)
    end
    return (i..., count)
end

hcubature_count (generic function with 1 method)

In [89]:
# non-singular example 
hcubature(x -> x[1]/norm(x)^3, (1,1,1), (2,2,2))

(0.08562994005985379, 1.2758662568828662e-9)

In [90]:
# non-singular example with count 
hcubature_count(x -> x[1]/norm(x)^3, (1,1,1), (2,2,2))[3]

10659

In [91]:
# singular example 
hcubature(x -> x[1]/norm(x)^3, (0,0,0), (1,1,1))

(0.9693880526621552, 1.4444861385160079e-8)

In [114]:
# singular example with count 
hcubature_count(x -> 1/norm(x)^3, (0,0,0), (1,1,1))

(Inf, NaN, 67287)

In [119]:
# singular example with count - constant magnetization - m(y) = a 
hcubature_count(x -> x[1]/norm(x)^3, (0,0,0), (1,1,1))

(0.9693880526621552, 1.4444861385160079e-8, 1098471)

In [117]:
# singular example with count - linear magnetization - m(y) = a * y + b 
hcubature_count(x -> ((x[1]+1)*x[1])/norm(x)^3, (0,0,0), (1,1,1))

(1.366067613186112, 2.0355841500269953e-8, 943965)

In [120]:
# second singular example with count - homogeneous linear magn - m(y) = a * y 
hcubature_count(x -> (x[1]*x[1])/norm(x)^3, (0,0,0), (1,1,1))

(0.396679560641997, 5.910556231655236e-9, 427911)

## Section 1: Define Vector Potential Density  

In [93]:
# define vector potential density 
function vp_density(r, rprime, M)
    num = cross(M, (r - rprime))
    denom = norm(r - rprime)^3
    result = num/denom
    return result   
end

vp_density (generic function with 1 method)

In [94]:
M = [0.,0.,1.]
r = [1.,0.,0.]
rp = [0.,1.,0.]
vp_density(r,rp,M)

3-element Vector{Float64}:
  0.3535533905932737
  0.3535533905932737
 -0.0

In [95]:
M = [0.,0.,1.]
r = [1.,0.,0.]
t = 0.5 
hcubature(rp -> vp_density(r,rp,M), (0.0, 0.0, -t), (1.0, 1.0, t))[1]

3-element Vector{Float64}:
 1.551694097302116
 1.5516940972927296
 0.0

In [96]:
M = [0.,0.,1.]
r = [1.,0.,0.]
t = 0.5 
hcubature_count(rp -> vp_density(r,rp,M), (0.0, 0.0, -t), (1.0, 1.0, t))

([1.551694097302116, 1.5516940972927296, 0.0], 3.26994515627534e-8, 2377155)

## Section 2: Perform Integration over Mesh 

In [17]:
# loop over midpoints of elements 
N = 5; h = 1/N;
xmesh = Vector(0:h:1)
ymesh = Vector(0:h:1)
for xi in xmesh[1:end-1] 
    for yj in ymesh[1:end-1] 
      mid = Point3D(xi+h/2,yj+h/2,0)
      # display(mid)
   end 
end 

In [52]:
# test integration assuming integrand for which answer can be checked analytically 
function settoone(rp)
    return 1.0
end 

N = 5; h = 1/N;
xmesh = Vector(0:h:1)
ymesh = Vector(0:h:1)
result = zeros(N,N)
for (i,xi) in enumerate(xmesh[1:end-1]) 
    for (j,yj) in enumerate(ymesh[1:end-1])  
      mid = Point3D(xi+h/2,yj+h/2,0)
      result[i,j] = hcubature(rp -> settoone(rp), (xi, yj, -t), (xi+h, yj+h, t))[1]
    end 
end

In [49]:
result

5×5 Matrix{Float64}:
 0.04  0.04  0.04  0.04  0.04
 0.04  0.04  0.04  0.04  0.04
 0.04  0.04  0.04  0.04  0.04
 0.04  0.04  0.04  0.04  0.04
 0.04  0.04  0.04  0.04  0.04

In [109]:
# assume r to coincide with midpoint of the element
N = 5; h = 1/N;
xmesh = Vector(0:h:1)
ymesh = Vector(0:h:1)
result = fill(zeros(3),N,N)
for (i,xi) in enumerate(xmesh[1:end-1]) 
    for (j,yj) in enumerate(ymesh[1:end-1])  
      mid = Point3D(xi,yj,-t)
      result[i,j] = hcubature(rp -> vp_density(mid,rp,M), (xi, yj, -t), (xi+h, yj+h, t))[1]
    end 
end

0.0

0.0

0.2

0.4

0.6

...

0.0

0.2

0.4

0.6

0.8

In [107]:
result

5×5 Matrix{Vector{Float64}}:
 [0.224443, -0.224443, 0.0]  …  [0.224443, -0.224443, 0.0]
 [0.224443, -0.224443, 0.0]     [0.224443, -0.224443, 0.0]
 [0.224443, -0.224443, 0.0]     [0.224443, -0.224443, 0.0]
 [0.224443, -0.224443, 0.0]     [0.224443, -0.224443, 0.0]
 [0.224443, -0.224443, 0.0]     [0.224443, -0.224443, 0.0]

In [58]:
# assume r to coincide with midpoint of the element
N = 5; h = 1/N;
xmesh = Vector(0:h:1)
ymesh = Vector(0:h:1)
result = fill(zeros(3),N,N)
for (i,xi) in enumerate(xmesh[1:end-1]) 
    for (j,yj) in enumerate(ymesh[1:end-1])  
      r = Point3D(xi,yj,-t)
      result[i,j] = hcubature_count(rp -> vp_density(r,rp,M), (xi, yj, -t), (xi+h, yj+h, t))[1]
    end 
end

In [59]:
result

5×5 Matrix{Vector{Float64}}:
 [0.224443, -0.224443, 0.0]  …  [0.224443, -0.224443, 0.0]
 [0.224443, -0.224443, 0.0]     [0.224443, -0.224443, 0.0]
 [0.224443, -0.224443, 0.0]     [0.224443, -0.224443, 0.0]
 [0.224443, -0.224443, 0.0]     [0.224443, -0.224443, 0.0]
 [0.224443, -0.224443, 0.0]     [0.224443, -0.224443, 0.0]

In [67]:
# assume r to coincide with midpoint of the element
N = 5; h = 1/N;
xmesh = Vector(0:h:1)
ymesh = Vector(0:h:1)
result = zeros(Int64,N,N)
for (i,xi) in enumerate(xmesh[1:end-1]) 
    for (j,yj) in enumerate(ymesh[1:end-1])  
      r = Point3D(xi,yj,-t)
      display(r)
      result[i,j] = hcubature_count(rp -> vp_density(r,rp,M), (xi, yj, -t), (xi+h, yj+h, t))[3]
    end 
end

3-element SVector{3, Float64} with indices SOneTo(3):
  0.0
  0.0
 -0.5

3-element SVector{3, Float64} with indices SOneTo(3):
  0.0
  0.2
 -0.5

3-element SVector{3, Float64} with indices SOneTo(3):
  0.0
  0.4
 -0.5

3-element SVector{3, Float64} with indices SOneTo(3):
  0.0
  0.6
 -0.5

3-element SVector{3, Float64} with indices SOneTo(3):
  0.0
  0.8
 -0.5

...

3-element SVector{3, Float64} with indices SOneTo(3):
  0.8
  0.0
 -0.5

3-element SVector{3, Float64} with indices SOneTo(3):
  0.8
  0.2
 -0.5

3-element SVector{3, Float64} with indices SOneTo(3):
  0.8
  0.4
 -0.5

3-element SVector{3, Float64} with indices SOneTo(3):
  0.8
  0.6
 -0.5

3-element SVector{3, Float64} with indices SOneTo(3):
  0.8
  0.8
 -0.5

In [66]:
result

5×5 Matrix{Int64}:
 1265781  1265781  1265781  1265781  1265781
 1265781  1265781  1265781  1265781  1265781
 1265781  1265781  1265781  1265781  1265781
 1265781  1265781  1265781  1265781  1265781
 1265781  1265781  1265781  1265781  1265781